In [70]:
import pandas as pd
import numpy as np

In [2]:
xls_CDNB = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/CDNB_cleaned.xlsx')
df_NB = pd.read_excel(xls_CDNB)
xls_NLQ = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/NLQ_cleaned.xlsx')
df_NLQ = pd.read_excel(xls_NLQ)
df_IT = df_NB.append(df_NLQ,ignore_index=True,sort=False) 

In [3]:
df_IT = df_IT.drop(df_IT[(df_IT.RegisterBuyVolume.isnull())&(df_IT.BuyVolume == 0)].index)
df_IT = df_IT.drop(df_IT[(df_IT.RegisterSellVolume.isnull())&(df_IT.SellVolume == 0)].index)

In [4]:
xls_1220 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2012_2020.xlsx')
xls_11 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2011.xlsx')
xls_21 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2021.xlsx')

df1_1220 = pd.read_excel(xls_1220, '1')
df2_1220 = pd.read_excel(xls_1220, '2')
df3_1220 = pd.read_excel(xls_1220, '3')
df4_11 = pd.read_excel(xls_11)
df5_21 = pd.read_excel(xls_21)

#Merge all into df_1121
df_1121 = df1_1220.append([df2_1220,df3_1220,df4_11,df5_21],ignore_index=True,sort=False)
df_1121_dup = df_1121.copy()
df_1121_dup.rename(columns={"TradingDate":"RegisterDate"}, inplace=True)

#Calculate daily ret of securities
df_1121_dup['Pi_ret'] = df_1121_dup.groupby('StockCode')['CloseAdj'].pct_change(-1)

In [5]:
#Map 'Exchange' & 'KLCPGD' values using cols
cols =['StockCode','RegisterDate']
df_IT['Exchange'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Exchange'])
df_IT['KLCPLH'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['KLCPLH'])

#Calculate date difference
from datetime import datetime
A = [d.date() for d in df_IT['DateStartExpected']]
B = [d.date() for d in df_IT['RegisterDate']]
df_IT['Diff_days'] = np.busday_count(B, A)

#Recalculate percentage values
df_IT['RegisterVolumeBeforePercent'] = df_IT['RegisterVolumeBefore']/df_IT['KLCPLH']
df_IT['RegisterVolumeAfterPercent'] = df_IT['RegisterVolumeAfter']/df_IT['KLCPLH']
df_IT['VolumeBeforePercent'] = df_IT['VolumeBefore']/df_IT['KLCPLH']
df_IT['VolumeAfterPercent'] = df_IT['VolumeAfter']/df_IT['KLCPLH']

#Cleaning data: drop all percentage values>1
df_IT = df_IT[(df_IT['RegisterVolumeBeforePercent'] < 1) &
               (df_IT['RegisterVolumeAfterPercent'] < 1) &
               (df_IT['VolumeBeforePercent'] < 1) & (df_IT['VolumeAfterPercent']<1)]

In [6]:
#Match Securities Ret

df_IT['Ri_T1_0'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Pi_ret'])

for x in range(1,13):
    df_IT['Ri_T1_' + str(x)] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols).groupby('StockCode')['Pi_ret'].shift(x))

In [7]:
df_IT.to_excel(r'/Users/hongnhungvu/Desktop/preview_13_all.xlsx',index=False)

In [71]:
xls_new = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/preview_13_all.xlsx')
df_new = pd.read_excel(xls_new)

In [72]:
df_new_buy = pd.DataFrame(df_new[(df_new['RegisterSellVolume'].isnull())])
df_new_sell = pd.DataFrame(df_new[(df_new['RegisterBuyVolume'].isnull())])

In [73]:
for i in range(len(df_new_buy)):
    k = df_new_buy['Diff_days'].iloc[i]
    df_new_buy.iloc[i,32+k] = (df_new_buy.iloc[i,32+k]*(-1))*(k/3)
    df_new_buy.iloc[i,32+k+1] = (df_new_buy.iloc[i,32+k+1]*(-1))*(k/3)
    df_new_buy.iloc[i,32+k+2] = (df_new_buy.iloc[i,32+k+2]*(-1))*(k/3)
df_new_buy

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,Ri_T1_3,Ri_T1_4,Ri_T1_5,Ri_T1_6,Ri_T1_7,Ri_T1_8,Ri_T1_9,Ri_T1_10,Ri_T1_11,Ri_T1_12
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,0.018272,...,-0.047619,-0.011364,-0.022472,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,0.032308,...,-0.035088,-0.025424,0.024793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PNC,2012-02-03,2012-02-17,2012-02-08,2012-03-08,2012-02-08,2012-02-15,GD CD noi bo,310000,0.031575,...,-0.043478,-0.041667,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LSS,2012-02-13,2012-02-17,2012-02-16,2012-02-28,2012-02-16,2012-02-16,GD CD noi bo,23462,0.000469,...,-0.009302,-0.018433,-0.040724,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DDM,2012-02-07,2012-02-20,2012-02-10,2012-03-31,2012-02-10,2012-02-16,GD CD noi bo,0,0.000000,...,0.034483,-0.000000,0.035714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,TVC,2020-12-11,2020-12-23,2020-12-14,2021-01-08,2020-12-14,2020-12-21,GD cua nguoi lien quan,16104073,0.349739,...,0.002710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11910,PGC,2020-11-19,2020-12-24,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,5066456,0.083966,...,-0.008969,0.002950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11911,DTL,2020-12-11,2020-12-24,2020-12-17,2021-01-16,2020-12-23,2020-12-24,GD cua nguoi lien quan,235000,0.003876,...,0.068814,-0.093151,-0.092190,-0.092615,NaN,NaN,NaN,NaN,NaN,NaN
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,-0.000000,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
ret_buy = np.nansum(df_new_buy.iloc[:,32:45],axis=1)
ret_buy

array([-0.0932193 , -0.02641077,  0.00086165, ..., -0.001954  ,
        0.12403101,  0.00273948])

In [75]:
df_new_buy['ret_buy'] = np.array(ret_buy)
df_new_buy

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,Ri_T1_4,Ri_T1_5,Ri_T1_6,Ri_T1_7,Ri_T1_8,Ri_T1_9,Ri_T1_10,Ri_T1_11,Ri_T1_12,ret_buy
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,0.018272,...,-0.011364,-0.022472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.093219
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,0.032308,...,-0.025424,0.024793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.026411
7,PNC,2012-02-03,2012-02-17,2012-02-08,2012-03-08,2012-02-08,2012-02-15,GD CD noi bo,310000,0.031575,...,-0.041667,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000862
8,LSS,2012-02-13,2012-02-17,2012-02-16,2012-02-28,2012-02-16,2012-02-16,GD CD noi bo,23462,0.000469,...,-0.018433,-0.040724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.077444
9,DDM,2012-02-07,2012-02-20,2012-02-10,2012-03-31,2012-02-10,2012-02-16,GD CD noi bo,0,0.000000,...,-0.000000,0.035714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.142948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,TVC,2020-12-11,2020-12-23,2020-12-14,2021-01-08,2020-12-14,2020-12-21,GD cua nguoi lien quan,16104073,0.349739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032634
11910,PGC,2020-11-19,2020-12-24,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,5066456,0.083966,...,0.002950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055263
11911,DTL,2020-12-11,2020-12-24,2020-12-17,2021-01-16,2020-12-23,2020-12-24,GD cua nguoi lien quan,235000,0.003876,...,-0.093151,-0.092190,-0.092615,NaN,NaN,NaN,NaN,NaN,NaN,-0.001954
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.124031


In [83]:
import scipy

from scipy.stats import wilcoxon

w, p = wilcoxon(df_new_buy['ret_buy'], alternative='greater')
w, p

(6462815.5, 3.1582825551923375e-12)

In [84]:
from scipy import stats
stats.norm.isf(p/2)

6.970494773915284

In [85]:
result = scipy.stats.ttest_1samp(df_new_buy['ret_buy'],0, alternative='greater')
result

Ttest_1sampResult(statistic=4.5602573684447485, pvalue=2.6122423985066646e-06)

In [86]:
import scipy.integrate 
from scipy.stats import norm
from scipy import stats

In [87]:
df_VNI_12 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2012)]

w_VNI_12, p_VNI_12 = wilcoxon(df_VNI_12['ret_buy'], alternative='greater')
w_VNI_12, p_VNI_12, norm.ppf(p_VNI_12), df_VNI_12['ret_buy'].median()

(30546.0, 6.022765784647685e-05, -3.8451976316500605, 0.014238804408766681)

In [88]:
df_VNI_13 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2013)]

w_VNI_13, p_VNI_13 = wilcoxon(df_VNI_13['ret_buy'], alternative='greater')
w_VNI_13, p_VNI_13, norm.ppf(p_VNI_13), df_VNI_13['ret_buy'].median()

(18876.0, 0.02694023937274571, -1.9277962146968055, 0.0072035226320490385)

In [89]:
df_VNI_14 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2014)]

w_VNI_14, p_VNI_14 = wilcoxon(df_VNI_14['ret_buy'], alternative='greater')
w_VNI_14, p_VNI_14, norm.ppf(p_VNI_14), df_VNI_14['ret_buy'].median()

(19288.0, 0.09116034519864391, -1.3336435889783345, 0.0021929624662882213)

In [90]:
df_VNI_15 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2015)]

w_VNI_15, p_VNI_15 = wilcoxon(df_VNI_15['ret_buy'], alternative='greater')
w_VNI_15, p_VNI_15, norm.ppf(p_VNI_15), df_VNI_15['ret_buy'].median()

(18131.0, 0.19814328562847627, -0.8482718853626056, 0.0013951051634961097)

In [91]:
df_VNI_16 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2016)]

w_VNI_16, p_VNI_16 = wilcoxon(df_VNI_16['ret_buy'], alternative='greater')
w_VNI_16, p_VNI_16, norm.ppf(p_VNI_16), df_VNI_16['ret_buy'].median()

(26867.0, 0.0014950194025427433, -2.968760152717517, 0.007499657810372769)

In [92]:
df_VNI_17 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2017)]

w_VNI_17, p_VNI_17 = wilcoxon(df_VNI_17['ret_buy'], alternative='greater')
w_VNI_17, p_VNI_17, norm.ppf(p_VNI_17), df_VNI_17['ret_buy'].median()

(34239.0, 0.0026319496888329258, -2.7904243875205665, 0.0015621329574818654)

In [93]:
df_VNI_18 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2018)]

w_VNI_18, p_VNI_18 = wilcoxon(df_VNI_18['ret_buy'], alternative='greater')
w_VNI_18, p_VNI_18, norm.ppf(p_VNI_18), df_VNI_18['ret_buy'].median()

(61924.5, 0.04280730609419932, -1.718998680505571, 0.002425210331487682)

In [94]:
df_VNI_19 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2019)]

w_VNI_19, p_VNI_19 = wilcoxon(df_VNI_19['ret_buy'], alternative='greater')
w_VNI_19, p_VNI_19, norm.ppf(p_VNI_19), df_VNI_19['ret_buy'].median()

(68286.5, 0.006045953918488883, -2.5094507432391353, 0.002218915422405378)

In [95]:
df_VNI_20 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2020)]

w_VNI_20, p_VNI_20 = wilcoxon(df_VNI_20['ret_buy'], alternative='greater')
w_VNI_20, p_VNI_20, norm.ppf(p_VNI_20), df_VNI_20['ret_buy'].median()

(57752.0, 0.527250806059637, 0.06836084772154302, 0.002387771893430087)

In [96]:
df_HNX_12 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2012)]

w_HNX_12, p_HNX_12 = wilcoxon(df_HNX_12['ret_buy'], alternative='greater')
w_HNX_12, p_HNX_12, norm.ppf(p_HNX_12), df_HNX_12['ret_buy'].median()

(351.0, 0.7068635274802008, 0.5442449188054672, 0.0)

In [97]:
df_HNX_13 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2013)]

w_HNX_13, p_HNX_13 = wilcoxon(df_HNX_13['ret_buy'], alternative='greater')
w_HNX_13, p_HNX_13, norm.ppf(p_HNX_13), df_HNX_13['ret_buy'].median()

(2034.0, 0.0932194131642603, -1.3211875805539017, 0.012754517614606109)

In [98]:
df_HNX_14 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2014)]

w_HNX_14, p_HNX_14 = wilcoxon(df_HNX_14['ret_buy'], alternative='greater')
w_HNX_14, p_HNX_14, norm.ppf(p_HNX_14), df_HNX_14['ret_buy'].median()

(6991.0, 0.006339571322113845, -2.4926565408914096, 0.014140193029267084)

In [99]:
df_HNX_15 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2015)]

w_HNX_15, p_HNX_15 = wilcoxon(df_HNX_15['ret_buy'], alternative='greater')
w_HNX_15, p_HNX_15, norm.ppf(p_HNX_15), df_HNX_15['ret_buy'].median()

(9783.0, 0.00031079410964094436, -3.4220152727700635, 0.01572456575682385)

In [100]:
df_HNX_16 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2016)]

w_HNX_16, p_HNX_16 = wilcoxon(df_HNX_16['ret_buy'], alternative='greater')
w_HNX_16, p_HNX_16, norm.ppf(p_HNX_16), df_HNX_16['ret_buy'].median()

(4200.0, 0.8401210780044408, 0.9949556313977334, 0.0)

In [101]:
df_HNX_17 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2017)]

w_HNX_17, p_HNX_17 = wilcoxon(df_HNX_17['ret_buy'], alternative='greater')
w_HNX_17, p_HNX_17, norm.ppf(p_HNX_17), df_HNX_17['ret_buy'].median()

(2904.0, 0.09626536783726936, -1.3031289671555086, 0.0)

In [102]:
df_HNX_18 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2018)]

w_HNX_18, p_HNX_18 = wilcoxon(df_HNX_18['ret_buy'], alternative='greater')
w_HNX_18, p_HNX_18, norm.ppf(p_HNX_18), df_HNX_18['ret_buy'].median()

(1340.0, 0.9343575426683431, 1.50905426966988, 0.0)

In [103]:
df_HNX_19 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2019)]

w_HNX_19, p_HNX_19 = wilcoxon(df_HNX_19['ret_buy'], alternative='greater')
w_HNX_19, p_HNX_19, norm.ppf(p_HNX_19), df_HNX_19['ret_buy'].median()

(3046.0, 0.013945218427861394, -2.198824034029873, 0.0027887831197997226)

In [104]:
df_HNX_20 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2020)]

w_HNX_20, p_HNX_20 = wilcoxon(df_HNX_20['ret_buy'], alternative='greater')
w_HNX_20, p_HNX_20, norm.ppf(p_HNX_20), df_HNX_20['ret_buy'].median()

(2468.0, 0.016916377715109964, -2.1220593380510855, 0.001937783411227155)

In [105]:
df_UPCOM_16 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2016)]

w_UPCOM_16, p_UPCOM_16 = wilcoxon(df_UPCOM_16['ret_buy'], alternative='greater')
w_UPCOM_16, p_UPCOM_16, norm.ppf(p_UPCOM_16), df_UPCOM_16['ret_buy'].median()

(2027.0, 0.8261649749516271, 0.9391182210525967, 0.0)

In [106]:
df_UPCOM_17 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2017)]

w_UPCOM_17, p_UPCOM_17 = wilcoxon(df_UPCOM_17['ret_buy'], alternative='greater')
w_UPCOM_17, p_UPCOM_17, norm.ppf(p_UPCOM_17), df_UPCOM_17['ret_buy'].median()

(4756.5, 0.159657753489726, -0.9958654857228307, 0.0)

In [107]:
df_UPCOM_18 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2018)]

w_UPCOM_18, p_UPCOM_18 = wilcoxon(df_UPCOM_18['ret_buy'], alternative='greater')
w_UPCOM_18, p_UPCOM_18, norm.ppf(p_UPCOM_18), df_UPCOM_18['ret_buy'].median()

(3962.5, 0.9159357676501643, 1.3782423860264044, 0.0)

In [108]:
df_UPCOM_19 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2019)]

w_UPCOM_19, p_UPCOM_19 = wilcoxon(df_UPCOM_19['ret_buy'], alternative='greater')
w_UPCOM_19, p_UPCOM_19, norm.ppf(p_UPCOM_19), df_UPCOM_19['ret_buy'].median()

(4295.0, 0.04079084641472097, -1.741581545740093, 0.0)

In [109]:
df_UPCOM_20 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2020)]

w_UPCOM_20, p_UPCOM_20 = wilcoxon(df_UPCOM_20['ret_buy'], alternative='greater')
w_UPCOM_20, p_UPCOM_20, norm.ppf(p_UPCOM_20), df_UPCOM_20['ret_buy'].median()

(4219.0, 0.04262071146431885, -1.7210518092387046, 0.0)

In [110]:
tstat_VNI_12, pval_VNI_12 = scipy.stats.ttest_1samp(df_VNI_12['ret_buy'],0, alternative='greater')
tstat_VNI_12, pval_VNI_12, df_VNI_12['ret_buy'].mean(), df_VNI_12['ret_buy'].std()

(4.209144216115575,
 1.678052081908336e-05,
 0.018277192974003213,
 0.07682239808257368)

In [111]:
tstat_VNI_13, pval_VNI_13 = scipy.stats.ttest_1samp(df_VNI_13['ret_buy'],0, alternative='greater')
tstat_VNI_13, pval_VNI_13, df_VNI_13['ret_buy'].mean(), df_VNI_13['ret_buy'].std()

(1.282904790675606,
 0.10033835883936959,
 0.00786835609155133,
 0.09870507011196651)

In [112]:
tstat_VNI_14, pval_VNI_14 = scipy.stats.ttest_1samp(df_VNI_14['ret_buy'],0, alternative='greater')
tstat_VNI_14, pval_VNI_14, df_VNI_14['ret_buy'].mean(), df_VNI_14['ret_buy'].std()

(2.023735504639636,
 0.022000426947098067,
 0.010147106470381122,
 0.08177664415542776)

In [113]:
tstat_VNI_15, pval_VNI_15 = scipy.stats.ttest_1samp(df_VNI_15['ret_buy'],0, alternative='greater')
tstat_VNI_15, pval_VNI_15, df_VNI_15['ret_buy'].mean(), df_VNI_15['ret_buy'].std()

(-0.2018841870321187,
 0.5799183340697566,
 -0.0009092386745323642,
 0.07317749863892056)

In [114]:
tstat_VNI_16, pval_VNI_16 = scipy.stats.ttest_1samp(df_VNI_16['ret_buy'],0, alternative='greater')
tstat_VNI_16, pval_VNI_16, df_VNI_16['ret_buy'].mean(), df_VNI_16['ret_buy'].std()

(2.820237295759252,
 0.0025587147071483426,
 0.012885681676658238,
 0.07940086220657727)

In [115]:
tstat_VNI_17, pval_VNI_17 = scipy.stats.ttest_1samp(df_VNI_17['ret_buy'],0, alternative='greater')
tstat_VNI_17, pval_VNI_17, df_VNI_17['ret_buy'].mean(), df_VNI_17['ret_buy'].std()

(2.89452526575318,
 0.00202057783759374,
 0.012040496623414908,
 0.07703961707615521)

In [116]:
tstat_VNI_18, pval_VNI_18 = scipy.stats.ttest_1samp(df_VNI_18['ret_buy'],0, alternative='greater')
tstat_VNI_18, pval_VNI_18, df_VNI_18['ret_buy'].mean(), df_VNI_18['ret_buy'].std()

(1.6912535835909097,
 0.045721469854209114,
 0.005365496871683902,
 0.06928837972395631)

In [117]:
tstat_VNI_19, pval_VNI_19 = scipy.stats.ttest_1samp(df_VNI_19['ret_buy'],0, alternative='greater')
tstat_VNI_19, pval_VNI_19, df_VNI_19['ret_buy'].mean(), df_VNI_19['ret_buy'].std()

(3.4116352517510387,
 0.00034947223711052374,
 0.011512697938767793,
 0.07492693655999035)

In [118]:
tstat_VNI_20, pval_VNI_20 = scipy.stats.ttest_1samp(df_VNI_20['ret_buy'],0, alternative='greater')
tstat_VNI_20, pval_VNI_20, df_VNI_20['ret_buy'].mean(), df_VNI_20['ret_buy'].std()

(-0.8618241206113679,
 0.8053944565759334,
 -0.0037421264632075697,
 0.09562483186965749)

In [120]:
tstat_HNX_14, pval_HNX_14 = scipy.stats.ttest_1samp(df_HNX_14['ret_buy'],0, alternative='greater')
tstat_HNX_14, pval_HNX_14, df_HNX_14['ret_buy'].mean(), df_HNX_14['ret_buy'].std()

(2.1778149868654535,
 0.015460940748684811,
 0.018972022432870522,
 0.1091547083471458)

In [121]:
tstat_HNX_15, pval_HNX_15 = scipy.stats.ttest_1samp(df_HNX_15['ret_buy'],0, alternative='greater')
tstat_HNX_15, pval_HNX_15, df_HNX_15['ret_buy'].mean(), df_HNX_15['ret_buy'].std()

(2.2450961276535044,
 0.012948256751795636,
 0.01304748186326347,
 0.08094548433477969)

In [122]:
tstat_HNX_16, pval_HNX_16 = scipy.stats.ttest_1samp(df_HNX_16['ret_buy'],0, alternative='greater')
tstat_HNX_16, pval_HNX_16, df_HNX_16['ret_buy'].mean(), df_HNX_16['ret_buy'].std()

(-2.5801437304664416,
 0.9945617147485932,
 -0.0384522539139078,
 0.1794576196684789)

In [123]:
tstat_HNX_17, pval_HNX_17 = scipy.stats.ttest_1samp(df_HNX_17['ret_buy'],0, alternative='greater')
tstat_HNX_17, pval_HNX_17, df_HNX_17['ret_buy'].mean(), df_HNX_17['ret_buy'].std()

(1.8335178763906679,
 0.034633706258365024,
 0.01598595478506124,
 0.09470969441225782)

In [124]:
df_HNX_17 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2017)]

w_HNX_17, p_HNX_17 = wilcoxon(df_HNX_17['ret_buy'], alternative='greater')
w_HNX_17, p_HNX_17, norm.ppf(p_HNX_17), df_HNX_17['ret_buy'].median()

(2904.0, 0.09626536783726936, -1.3031289671555086, 0.0)

In [125]:
df_HNX_18 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2018)]

w_HNX_18, p_HNX_18 = wilcoxon(df_HNX_18['ret_buy'], alternative='greater')
w_HNX_18, p_HNX_18, norm.ppf(p_HNX_18), df_HNX_18['ret_buy'].median()

(1340.0, 0.9343575426683431, 1.50905426966988, 0.0)

In [126]:
df_HNX_19 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2019)]

w_HNX_19, p_HNX_19 = wilcoxon(df_HNX_19['ret_buy'], alternative='greater')
w_HNX_19, p_HNX_19, norm.ppf(p_HNX_19), df_HNX_19['ret_buy'].median()

(3046.0, 0.013945218427861394, -2.198824034029873, 0.0027887831197997226)

In [127]:
df_HNX_20 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2020)]

w_HNX_20, p_HNX_20 = wilcoxon(df_HNX_20['ret_buy'], alternative='greater')
w_HNX_20, p_HNX_20, norm.ppf(p_HNX_20), df_HNX_20['ret_buy'].median()

(2468.0, 0.016916377715109964, -2.1220593380510855, 0.001937783411227155)

In [128]:
df_UPCOM_16 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2016)]

w_UPCOM_16, p_UPCOM_16 = wilcoxon(df_UPCOM_16['ret_buy'], alternative='greater')
w_UPCOM_16, p_UPCOM_16, norm.ppf(p_UPCOM_16), df_UPCOM_16['ret_buy'].median()

(2027.0, 0.8261649749516271, 0.9391182210525967, 0.0)

In [129]:
df_UPCOM_17 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2017)]

w_UPCOM_17, p_UPCOM_17 = wilcoxon(df_UPCOM_17['ret_buy'], alternative='greater')
w_UPCOM_17, p_UPCOM_17, norm.ppf(p_UPCOM_17), df_UPCOM_17['ret_buy'].median()

(4756.5, 0.159657753489726, -0.9958654857228307, 0.0)

In [130]:
df_UPCOM_18 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2018)]

w_UPCOM_18, p_UPCOM_18 = wilcoxon(df_UPCOM_18['ret_buy'], alternative='greater')
w_UPCOM_18, p_UPCOM_18, norm.ppf(p_UPCOM_18), df_UPCOM_18['ret_buy'].median()

(3962.5, 0.9159357676501643, 1.3782423860264044, 0.0)

In [131]:
df_UPCOM_19 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2019)]

w_UPCOM_19, p_UPCOM_19 = wilcoxon(df_UPCOM_19['ret_buy'], alternative='greater')
w_UPCOM_19, p_UPCOM_19, norm.ppf(p_UPCOM_19), df_UPCOM_19['ret_buy'].median()

(4295.0, 0.04079084641472097, -1.741581545740093, 0.0)

In [132]:
df_UPCOM_20 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2020)]

w_UPCOM_20, p_UPCOM_20 = wilcoxon(df_UPCOM_20['ret_buy'], alternative='greater')
w_UPCOM_20, p_UPCOM_20, norm.ppf(p_UPCOM_20), df_UPCOM_20['ret_buy'].median()

(4219.0, 0.04262071146431885, -1.7210518092387046, 0.0)

In [133]:
tstat_UPCOM_16, pval_UPCOM_16 = scipy.stats.ttest_1samp(df_UPCOM_16['ret_buy'],0, alternative='greater')
tstat_UPCOM_16, pval_UPCOM_16, df_UPCOM_16['ret_buy'].mean(), df_UPCOM_16['ret_buy'].std()

(-1.1541177210491402,
 0.8746552867041749,
 -0.018113295131693137,
 0.17476650308425432)

In [134]:
tstat_UPCOM_17, pval_UPCOM_17 = scipy.stats.ttest_1samp(df_UPCOM_17['ret_buy'],0, alternative='greater')
tstat_UPCOM_17, pval_UPCOM_17, df_UPCOM_17['ret_buy'].mean(), df_UPCOM_17['ret_buy'].std()

(1.0811871160267659,
 0.14054238845646394,
 0.008228814887165767,
 0.10154216808825674)

In [135]:
tstat_UPCOM_18, pval_UPCOM_18 = scipy.stats.ttest_1samp(df_UPCOM_18['ret_buy'],0, alternative='greater')
tstat_UPCOM_18, pval_UPCOM_18, df_UPCOM_18['ret_buy'].mean(), df_UPCOM_18['ret_buy'].std()

(-1.9701366112252845,
 0.9748505140822481,
 -0.02425423761628633,
 0.16879913275648423)

In [136]:
tstat_UPCOM_19, pval_UPCOM_19 = scipy.stats.ttest_1samp(df_UPCOM_19['ret_buy'],0, alternative='greater')
tstat_UPCOM_19, pval_UPCOM_19, df_UPCOM_19['ret_buy'].mean(), df_UPCOM_19['ret_buy'].std()

(1.917532401030221,
 0.028363828241087588,
 0.019910328154519586,
 0.14084603268629006)

In [137]:
tstat_UPCOM_20, pval_UPCOM_20 = scipy.stats.ttest_1samp(df_UPCOM_20['ret_buy'],0, alternative='greater')
tstat_UPCOM_20, pval_UPCOM_20, df_UPCOM_20['ret_buy'].mean(), df_UPCOM_20['ret_buy'].std()

(1.6915300572859495,
 0.046099050628333,
 0.013778753456879565,
 0.1191618115032544)